In [1]:
import pandas as pd
import datetime
import numpy as np
import random
import seaborn as sn
from pandas_profiling import ProfileReport
from tenzing.core.summary import get_template, traverse_config, summary_report
from tenzing.core.model_implementations.typesets import tenzing_standard, tenzing_geometry_set

%load_ext autoreload
%autoreload 2


In [2]:
df = pd.DataFrame({'item_id': [1, 1, 3], 
                   'cost': [2.1, 3.5, 4], 
                   'item': ['orange', 'orange', 'apple'],
                   'sale_date': pd.to_datetime([datetime.date(2011,1,1), datetime.date(2012, 1, 1), datetime.date(2013,1,1)]),
                   'store_geography': pd.Series(['POINT (12 42)', 'POINT (100 42.723)', 'POINT (0 0)']),
                   'cogs': pd.Series([np.nan, 1.1, 2.1]).astype(str),
                   'still_available': [True, False, True],
                   'expired': ['True', 'false', 'False']
                   })


In [4]:
ts = tenzing_geometry_set()
ts.prep(df)
df = ts.cast_to_inferred_types(df)

ts = tenzing_geometry_set()
ts.prep(df)
summer = ts.summary_report(df)
summer

Number of Observations,3
Number of Variables,8
tenzing_integer,1
tenzing_float,2
tenzing_string,1
tenzing_timestamp,1
tenzing_geometry,1
tenzing_bool,2
nunique,2.0
mean,1.67
std,1.15


In [4]:
ProfileReport(df)

Number of variables,8
Number of observations,3
Total Missing (%),0.0%
Total size in memory,230.0 B
Average record size in memory,76.7 B
Numeric,1
Categorical,1
Boolean,2
Date,1
Text (Unique),0
Rejected,2


In [12]:
import geopandas as gpd
from geopandas.datasets import naturalearth_cities
path = gpd.datasets.get_path('naturalearth_lowres')
df = gpd.read_file(path)

ts = tenzing_geometry_set()
ts.prep(df)
ts.summary_report(df)

Number of Observations,177
Number of Variables,6
tenzing_integer,1
tenzing_string,3
tenzing_float,1
tenzing_geometry,1
nunique,177.0
mean,41712369.84
std,145969090.84
max,1379302771.0
min,140.0


In [15]:
gpd.datasets.available

['naturalearth_cities', 'naturalearth_lowres', 'nybb']

In [33]:
class html_repr:
    def __init__(self, html):
        self.html = html
    
    def _repr_html_(self):
        return self.html
    
row_temp = '<tr class="{{ extra_class }}">\
        <td class="fillremaining">{{ label }}</td>\
        <td class="number">{{ count }}</td>\
        <td class="number">{{ percentage }}%</td>\
        <td>\
            <div class="bar" style="width:{{ width }}%">&nbsp;</div>\
        </td>\
</tr>'\

table_temp = '<table class="freq table table-hover">\
    <thead>\
    <tr>\
        <td class="fillremaining">Value </td>\
        <td class="number"> Count </td>\
        <td class="number">Frequency (%)</td>\
        <td style="min-width:200px">&nbsp;</td>\
    </tr>\
    </thead>\
    {{ rows }}\
</table>'


from jinja2 import Environment, BaseLoader

rowTemp = Environment(loader=BaseLoader).from_string(row_temp)
tableTemp = Environment(loader=BaseLoader).from_string(table_temp)

data = [{'label': 'a', 'count': 2, 'percentage': 50}, {'label': 'b', 'count': 2, 'percentage': 50}]
row_html = ''
for dat in data:
    row_html += rowTemp.render(**dat)
res = tableTemp.render(rows=row_html)
html_repr(res)

Value,Count,Frequency (%),
a,2,50%,
b,2,50%,


In [98]:
row_temp = '<table class="freq table table-hover">\
    <thead>\
    <tr>\
        <td class="fillremaining">Value </td>\
        <td class="number"> Count </td>\
        <td class="number">Frequency (%)</td>\
        <td style="min-width:200px">&nbsp;</td>\
    </tr>\
    </thead>\
    {% for key, count in data.items() %}\
        <tr class="{{ extra_class }}">\
                <td class="fillremaining">{{ key }}</td>\
                <td class="number">{{ count }}</td>\
                <td class="number">{{((count / total) * 10000 // 1) / 100  }}%</td>\
                <td>\
                    <div class="bar" style="width:{{ (count / total * 100) // 1 }}%">&nbsp;</div>\
                </td>\
        </tr>\
    {% endfor %}\
</table>'
data = {'a': 2, 'b': 4, 'c': 4, 'd': 1}
rowTemp = Environment(loader=BaseLoader).from_string(row_temp)
row_html = rowTemp.render(data=data, total=sum(val for val in data.values()))
res = row_html
html_repr(res)

Value,Count,Frequency (%),
a,2,18.18%,
b,4,36.36%,
c,4,36.36%,
d,1,9.09%,


In [107]:
from tenzing.core.summary import create_frequency_table

html_repr(create_frequency_table(data))

Value,Count,Frequency (%),
b,4,36.36%,
c,4,36.36%,
a,2,18.18%,
d,1,9.09%,


In [104]:
import heapq
heapq.nlargest(6, data, key=data.get)

['b', 'c', 'a', 'd']

In [80]:
%timeit int((2/3) * 1000)

152 ns ± 4.49 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


In [81]:
%timeit (2/3) * 1000 // 1

9.85 ns ± 0.18 ns per loop (mean ± std. dev. of 7 runs, 100000000 loops each)
